In [1]:
import sys
from opentree import OT

In [20]:
# OOh! We can mix togther python and bash commands :P
# The '!' at the start of the line means the command is executed in bash
# This wget command pulls GBIF data file from the internet, and saves it as 'gbif_example.csv'
!wget -O gbif_example.csv https://raw.githubusercontent.com/McTavishLab/biodiversity_next/master/example.csv

--2019-12-29 17:35:06--  http://api.gbif.org/v1/occurrence/download/request/0023200-190918142434337.zip
Resolving api.gbif.org (api.gbif.org)... 130.225.43.2
Connecting to api.gbif.org (api.gbif.org)|130.225.43.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37100446 (35M) [application/octet-stream]
Saving to: ‘gbif_example.zip’

gbif_example.zip    100%[===================>]  35.38M  3.22MB/s    in 17s     

2019-12-29 17:35:24 (2.09 MB/s) - ‘gbif_example.zip’ saved [37100446/37100446]

gzip: gbif_example.zip: unknown suffix -- ignored
example.csv    GA_waterfowl.tre      names.tre
example.csv.1  gbif_example.csv      OpenTree_demo.ipynb
example.csv.2  gbif_exampledata.csv  SSB2020_gbif_integration.ipynb
example.csv.3  gbif_example.zip
example.csv.4  gbif_taxa.tre


In [21]:
!head -n 3 gbif_example.csv
# Oof! Lots of information.

gbifID	datasetKey	occurrenceID	kingdom	phylum	class	order	family	genus	species	infraspecificEpithet	taxonRank	scientificName	verbatimScientificName	verbatimScientificNameAuthorship	countryCode	locality	stateProvince	occurrenceStatus	individualCount	publishingOrgKey	decimalLatitude	decimalLongitude	coordinateUncertaintyInMeters	coordinatePrecision	elevation	elevationAccuracy	depth	depthAccuracy	eventDate	day	month	year	taxonKey	speciesKey	basisOfRecord	institutionCode	collectionCode	catalogNumber	recordNumber	identifiedBy	dateIdentified	license	rightsHolder	recordedBy	typeStatus	establishmentMeans	lastInterpreted	mediaType	issue
2423004790	50c9509d-22c7-4a22-a47d-8c48425ef4a7	https://www.inaturalist.org/observations/32478397	Animalia	Arthropoda	Insecta	Odonata	Libellulidae	Tramea	Tramea lacerata		SPECIES	Tramea lacerata Hagen, 1861	Tramea lacerata		US		California			28eb1a3f-1c15-4a95-931a-4af90ecb574d	37.36422	-120.424003	12.0						2019-09-10T08:39:35Z	10	9	2019	1428475	1428475	HUMAN_OB

In [18]:
filename = "gbif_example.csv"
fi = open(filename)
header = fi.readline().split('\t') # Save the first line seperately as the header

gbif_data = fi.readlines() #read in the data

#Get indexes for each column in the csv file
col_dict = {}
for i, col in enumerate(header):
    col_dict[col] = i
    
# Would this make more sense to do in Pandas? Probably! But I like loops.

In [6]:
# Now we know what column each of out data types are in.
# So much (many) data!
col_dict

{'gbifID': 0,
 'datasetKey': 1,
 'occurrenceID': 2,
 'kingdom': 3,
 'phylum': 4,
 'class': 5,
 'order': 6,
 'family': 7,
 'genus': 8,
 'species': 9,
 'infraspecificEpithet': 10,
 'taxonRank': 11,
 'scientificName': 12,
 'verbatimScientificName': 13,
 'verbatimScientificNameAuthorship': 14,
 'countryCode': 15,
 'locality': 16,
 'stateProvince': 17,
 'occurrenceStatus': 18,
 'individualCount': 19,
 'publishingOrgKey': 20,
 'decimalLatitude': 21,
 'decimalLongitude': 22,
 'coordinateUncertaintyInMeters': 23,
 'coordinatePrecision': 24,
 'elevation': 25,
 'elevationAccuracy': 26,
 'depth': 27,
 'depthAccuracy': 28,
 'eventDate': 29,
 'day': 30,
 'month': 31,
 'year': 32,
 'taxonKey': 33,
 'speciesKey': 34,
 'basisOfRecord': 35,
 'institutionCode': 36,
 'collectionCode': 37,
 'catalogNumber': 38,
 'recordNumber': 39,
 'identifiedBy': 40,
 'dateIdentified': 41,
 'license': 42,
 'rightsHolder': 43,
 'recordedBy': 44,
 'typeStatus': 45,
 'establishmentMeans': 46,
 'lastInterpreted': 47,
 'medi

In [7]:
# As described in the TNRS, sction, 
# we can use OpenTree API's to match out Gbif identifiers to Open Tree unique identifiers

match_dict = {} # This will list the matches
ott_ids = set() # And generated a set of taxa

#Loop through each line in the gbif output
for lin in gbif_data:
    lii = lin.split('\t')
    gb_id = lii[col_dict['taxonKey']] # this grabs the gbif id number from the right column
    sys.stdout.write(".") #progress bar
    sys.stdout.flush()
    if gb_id in match_dict:
        #Skip gb_id's you have already matched
        pass
    else:
        # Do a direct match to gbif id's in the open tree taxonomy
        ott_id = OT.get_ottid_from_gbifid(gb_id)
        if ott_id == None:
            # Sometimes we don't have a record of the gbif ID, but we do have a taxon with that exact name
            # Search on the name
            spp_name = lii[col_dict['verbatimScientificName']]
            ott_id = OT.get_ottid_from_name(spp_name)
            if ott_id == None:
                sys.stdout.write("Couldn't find an id for {}, gbif {}".format(spp_name, gb_id))
        match_dict[gb_id] = ott_id
        ott_ids.add(ott_id)

...................................................................................................

In [9]:
# Lets grab a tree fo those taxa!
output = OT.synth_induced_tree(ott_ids=list(ott_ids),  label_format='name')
treefile = "gbif_taxa.tre"
output.tree.write(path = treefile, schema = "newick")
sys.stdout.write("Tree written to {}\n".format(treefile))

Tree written to gbif_taxa.tre


In [10]:
# if we print to string we can take a quick look over at icytree.org
output.tree.as_string(schema="newick")

"(((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((Regulus_calendula)Regulus)mrcaott246ott3599436,(((((((((Sturnus_vulgaris)mrcaott366470ott565813)mrcaott2224ott366470)mrcaott2175ott2224)mrcaott2175ott59905,((((((((Mimus_polyglottos)mrcaott571310ott1000585)mrcaott349284ott571310)mrcaott349284ott817639)mrcaott349284ott556541)mrcaott325207ott349284)mrcaott259082ott325207)mrcaott259082ott259086)mrcaott259082ott474281)mrcaott2175ott259082)mrcaott2175ott968664)mrcaott1566ott2175)mrcaott1566ott496009)mrcaott1566ott3598440)mrcaott246ott1566,(((((((Phainopepla_nitens)Phainopepla)mrcaott33427ott294590)mrcaott33427ott179330)mrcaott5934ott33427)mrcaott5934ott496029)mrcaott5934ott294597)mrcaott5934ott33425)mrcaott246ott5934,(((((((((((((((((((((((Passerculus_sandwichensis)Passerculus)mrcaott292636ott453758)mrcaott261498ott292636)mrcaott261498ott292635)mrcaott261498ott934463)mrcaott261498ott292639)mrcaott6023ott261498,(((((Zonotrichia_leucophrys)mrcaott125079ott265554)mrcaott1250

In [11]:
# Don't forget to cite your friendly phylogeneticists!
studies = output.response_dict['supporting_studies']
cites = OT.get_citations(studies) #this can be a bit slow
print(cites)